In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import time
from collections import defaultdict as dd
import pickle


#### Setting up teams to iterate through

In [3]:
teams = ['ARI', 'ATL', 'BAL', 'BOS', 'CHC', 'CHW', 'CIN', 'CLE', 'COL', 'DET', 'HOU', 'KCR',  'LAA', 'LAD', 'MIA', 'MIL', 'MIN', 'NYM', 'NYY', 'OAK', 'PHI', 'PIT', 'SDP', 'SEA', 'SFG', 'STL', 'TBR', 'TEX', 'TOR', 'WSN']

## Baseball Reference Scraping

In [291]:
url = 'https://www.baseball-reference.com/teams/ARI/2019-schedule-scores.shtml' 

response = requests.get(url)
soup = BeautifulSoup(response.text, "html5lib")
table = soup.find(class_ = "table_wrapper", id = "all_team_schedule")

headers = table.find_all('th')

In [292]:
col_names = []
iters = 0

for i in headers:
    if iters < 22:
        col_names.append(i.get("data-stat"))
        iters +=1
    else:
        break
    

In [293]:
def get_sched(team):
    url = 'https://www.baseball-reference.com/teams/'+team+'/2019-schedule-scores.shtml' 

    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html5lib")
    table = soup.find(class_ = "table_wrapper", id = "all_team_schedule")

    data = table.find_all('tr')
    
    rows = []

    for i in data:
        cols = i.find_all('td')
        cols2 = [ele.text.strip() for ele in cols]
        if cols2 != []:
            rows.append(cols2)
            
    counter = 0
    for i in rows:
        i.insert(0,counter)
        counter += 1
        
    return rows

In [294]:
total = []
for i in teams:
    total.append(get_sched(i))
    

In [295]:
all_games = []
for i in total:
    for game in i:
         all_games.append(game)

    

In [350]:
games_df = pd.DataFrame(all_games, columns = col_names)
games_df['team_game'] = games_df['team_game'] + 1
games_df['date'] = games_df['date_game'].str.rsplit(', ').str[-1] +', 2019'
games_df['DOW'] = games_df['date_game'].str.rsplit(', ').str[0]

games_df['date2'] = pd.to_datetime(games_df['date'], errors='coerce').astype(str)
games_df = games_df.drop('date', axis = 1)


# Weather Data Scrape (Weather Undergound)

In [4]:
urls = pd.read_csv('/Users/philliprichardson/Metis/Module 2/links.csv')

w_col_headers = ['City','Month', 'Day', 'max_temp', 'avg_temp', 'min_temp', 'max_dew', 'avg_dew', 'min_dew', 'max_humid', 'avg_humid', 'min_humid', 'max_wind', 'avg_wind','min_wind', 'max_pressure', 'avg_pressure', 'min_pressure', 'precip']
months = ['2019-3', '2019-4', '2019-5', '2019-6', '2019-7', '2019-8', '2019-9']
urls = urls[['Tm','Base']]
urls['Base'] = urls['Base'].apply(lambda x: x[:-6])
links = urls['Base'].tolist()
tms = urls['Tm'].tolist()

weather_data = pd.DataFrame(columns = w_col_headers)

#### Operationalizing the scraping

In [215]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver
    
def render_page(url):
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    time.sleep(3)
    r = driver.page_source
    driver.quit()
    return r
                

In [216]:
def month_weather(url, city_name):
    r = render_page(url)
    soup = BeautifulSoup(r, "html5lib")

    table = soup.find('lib-city-history-observation').find('tbody')
    

    weather = []

    for r in table.find_all('tr', class_='ng-star-inserted'):
        for i in r.find_all('td', class_='ng-star-inserted'):
            td = i.text
            td = td.strip('  ')
            weather.append(td)

    month = weather.pop(0)

    weather.append( 'end')
    weather.append( 'end')
  

    weath_row = []
    w_data = []
    keys = 1

    for idx, elem in enumerate(weather):
        
        if elem.replace(".","",1).replace("-", "", 1).isdigit() == True:
            w_data.append(elem)
        elif elem.replace(".","",1).replace("-", "", 1).isdigit() == False and weather[idx - 1].replace(".","",1).replace("-", "", 1).isdigit() == True:
            weath_row.append(w_data.copy())
            keys += 1
            w_data.clear()
        else: 
            pass
        
    day_chunks = []

    def divide_chunks(l, n):
        for i in range(0, len(l), n): 
            yield l[i:i + n]


    days = len(weath_row[0])

    city = [city_name] * days
    month_ = [month] * days

    for col in weath_row:
        day_chunks.append(divide_chunks(col, days))

    w_cols = [city, month_]

    for rec in day_chunks:
        for j in list(rec):
            w_cols.append(j)

    arr = np.array(w_cols)

    return pd.DataFrame(arr.T)

#### Manually iterating due to page crashes

In [274]:
# weather_dfs = []
# Manually Iterating to prevent overloading

#0-29
ctr = 0

input_team = urls.iloc[ctr]['Tm']
curr_url = urls.iloc[ctr]['Base']

for i in months:
    input_url = curr_url+i
    weather_dfs.append(month_weather(input_url, input_team))
    with open('weather.pkl', 'wb') as f:
        pickle.dump(weather_dfs, f)


In [275]:
#Finding Partial Scrapes indeces

remove_idx = []

for i, e in enumerate(weather_dfs):
    if e.shape[1] != 19:
        remove_idx.append(i)

In [276]:
# Removing partial scrapes from the list of all scrapes

for index in sorted(remove_idx, reverse=True):
    del weather_dfs[index]

In [277]:
for i in weather_dfs:
    i.columns = w_col_headers

full_wd = weather_data.append(weather_dfs)

full_wd = full_wd[['City', 'Month', 'Day', 'max_temp', 'avg_temp', 'precip']]

full_wd.drop_duplicates(inplace = True)

## Merging weather and baseball data

In [357]:
full_wd['Date'] = pd.to_datetime(full_wd['Month'] + ' ' + full_wd['Day'] +', ' + '2019').astype(str)


all_data = games_df.merge(full_wd, left_on = ['team_ID', 'date2'], right_on = ['City', 'Date'])

keep = ['team_game', 'team_ID', 'homeORvis', 'opp_ID', 'win_loss_result', 'R', 'RA', 'extra_innings', 'win_loss_record', 'games_back', 'day_or_night', 'attendance', 'win_loss_streak', 'reschedule', 'DOW', 'max_temp', 'avg_temp', 'precip', 'Date']
all_data = all_data[keep]

all_data.to_csv(r'/Users/philliprichardson/Metis/Module 2/modelbase.csv')
